In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
file = open('ecommerce_data.csv')
data = pd.read_csv(file, encoding='cp949')
data.head()

- InvoiceNo: 영수증번호
- StockCode: 상품번호
- Description: 상품명
- Quantity: 판매수량
- InvoiceDate: 결제날짜
- UnitPrice: 개당 가격
- CustomerID: 고객번호
- Country: 나라

#시간의 흐름에 따라 매출, 주문고객수, 주문단가의 추이는 어떻게 달라지는가?
#리텐션 분석 : 시간의 흐름에 따라 고객들은 얼마나 남고 얼마나 이탈했는가?
##- 시간에 따른 유저의 이탈과 잔존 분석
#RFM 분석 : 고객의 행동에 따라 고객을 유형화
##- Recency(최근성) : 고객이 얼마나 최근에 구매를 했는가
##- Frequency(빈도) : 고객이 얼마나 자주 구매를 했는가
##- Monetary(금액) : 구객이 구매한 총 금액

In [ ]:
data.info()

In [ ]:
data.dropna(subset=['CustomerID'], inplace=True)
data.info()

In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format = '%m/%d/%Y %H:%M')
data['CustomerID'] = data['CustomerID'].astype(int).astype(str)
data.info()

In [ ]:
data.head()

In [ ]:
data['date_ymd'] = data['InvoiceDate'].dt.date.astype('datetime64[ns]')
data['year'] = data['InvoiceDate'].dt.year
data.head()

In [ ]:
# 매출 칼럼
data['amount'] = data['Quantity'] * data['UnitPrice']
data.info()

In [ ]:
data.query('Quantity < 0')
#주문 취소의 경우

In [ ]:
#주문 취소한 데이터 제외하기
data = data.query('Quantity > 0')

In [ ]:
data.info()

In [ ]:
#시간의 흐름에 따라 매출의 추이는 어떻게 달리지는가
#매출
amount_by_date = data.groupby('date_ymd')[['amount']].sum().reset_index()
fig = px.line(data_frame=amount_by_date, x = 'date_ymd',y = 'amount')
fig.show()

In [ ]:
#주문 고객수
customer_count_by_date = data.groupby('date_ymd')[['CustomerID']].nunique().reset_index().rename({"CustomerID":"customer_count"}, axis = 1)
fig = px.line(data_frame = customer_count_by_date, x = 'date_ymd', y= 'customer_count')
fig.show()
#주문 고객수 우상향

In [ ]:
#주문단가 = 총매출 / 주문건수
#주문 건수
invoice_count_by_date = data.groupby('date_ymd')[['InvoiceNo']].nunique().reset_index().rename({'InvoiceNo':'invoice_count'}, axis = 1)
invoice_count_by_date.head()

In [ ]:
invoice_amount = pd.merge(amount_by_date, invoice_count_by_date, on='date_ymd')
invoice_amount['amount_per_invoice'] = invoice_amount['amount'] / invoice_amount['invoice_count']
invoice_amount.head()

In [ ]:
fig = px.line(data_frame=invoice_amount, x = 'date_ymd', y= 'amount_per_invoice')
fig.show()
#주문단가는 비교적 일정하게 유지되다가 일정한 날 주문 단가가 높았던 상품이 있었던 것으로 확인

In [ ]:
#시간의 흐름에 따라 고객들의 잔존과 이탈 분석
#연월 단위로 고객번호, 영수증 번호 전처리
retention_base = data[['CustomerID', 'InvoiceNo','date_ymd']].drop_duplicates()#겹치지 않게 빼오기
retention_base['date_ym'] = retention_base['date_ymd'].dt.to_period('M')#연월 칼럼
retention_base.head()

In [ ]:
print(min(data['date_ymd'].unique()))
print(max(data['date_ymd'].unique()))
#2011 12월 데이터를 포함하면 11월 리텐션이 낮아질 수 밖에 없다
#따라서 2011 12월 데이터는 제외

In [ ]:
retention_base = retention_base.query('date_ymd <= "2011-11-30"')

In [ ]:
date_ym_list = sorted(list(retention_base['date_ym'].unique()))

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
retention = pd.DataFrame()
for s in tqdm(date_ym_list):
    for t in date_ym_list:
        period_start = s
        period_target = t

        if period_start <= period_target:
            period_start_users = set(retention_base.query('date_ym == @period_start')['CustomerID'])
            period_target_users = set(retention_base.query('date_ym == @period_target')['CustomerID'])

            retained_users = period_start_users.intersection(period_target_users)

            retention_rate = len(retained_users) / len(period_start_users)

            temp = pd.DataFrame({'cohort':[period_start], 'date_ym':[period_target], 'retention_rate':[retention_rate]})

            retention = pd.concat([retention, temp])

In [ ]:
retention['cohort_size(month)'] = retention.apply(lambda x : (x['date_ym'] - x['cohort']).n, axis = 1)
retention.head()

In [ ]:
retention['cohort'] = retention['cohort'].astype(str)
retention['date_ym'] = retention['date_ym'].astype(str)

In [ ]:
retention_final = pd.pivot_table(data=retention, index='cohort', columns='cohort_size(month)', values='retention_rate')
retention_final

In [ ]:
fig = px.imshow(retention_final, text_auto='.2%', color_continuous_scale='Burg')
fig.show()
#최근일수록 retention 비율이 높다
#대각선으로 보면 2011년 11월 유저들이 많이 복귀함
#2011년 9월에도 많이 복귀함

In [ ]:
retention_curve = retention.groupby('cohort_size(month)')[['retention_rate']].mean().reset_index()
retention_curve

In [ ]:
fig = px.line(data_frame = retention_curve, x='cohort_size(month)', y='retention_rate', title='리텐션 커브')
fig.update_yaxes(tickformat='.2%')
fig.show()
#기간동안의 cohort_size에 따른 리텐션 커브의 평균을 확인
#retention이 cohort_size가 커지면 내겨라기 마련인데 유지하다가 최근에 오히려 높아진 것을 확인

In [ ]:
#RFM : 고객의 행동에 따른 고객 유형화
#데이터가 1년치밖에서 없어서 RM분석 진행
data.head()

In [ ]:
today_date = max(data['date_ymd'])

rfm = data.groupby('CustomerID').agg({'InvoiceDate': lambda x: (today_date - x.max()).days, #오늘로부터 며칠이 지났는지
                                    'amount': lambda x: x.sum()}) #주문금액

rfm.columns = ['recency', 'monetary']
rfm.head()

In [ ]:
#각 팩터를 5등급으로 나눠 등급을 매김
pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])

In [ ]:
rfm['recency_score'] = pd.qcut(rfm["recency"], 3, labels=[3, 2, 1])
rfm['monetary_score'] = pd.qcut(rfm["monetary"], 3, labels=[1, 2, 3])
rfm['rm_score'] = rfm['recency_score'].astype(str) + rfm['monetary_score'].astype(str)
rfm.reset_index(inplace=True)
rfm

In [ ]:
rm_score = rfm.groupby('rm_score')[['CustomerID']].nunique().reset_index().rename({'CustomerID':'customer_count'}, axis=1)
rm_score

In [ ]:
def categorize_customer(score):
    if score == '33':
        return '최우수' #최신성, 구매 모두 상당히 높음
    elif score in ['32','23','22']:
        return '우수' #최신성, 구매 모두 높음
    elif score =='11':
        return '휴면' #최신성, 구매 모두 낮음
    elif score in ['12','13']:
        return '이탈 방지' #구매는 높으나 최신성은 낮음 -> 다시 불러들어야 함
    elif score in ['31','21']:
        return '구매 유도' #최신성은 높으나 구매는 낮음 -> 구매를 유도해야 함

rm_score['category'] = rm_score['rm_score'].apply(categorize_customer)

In [ ]:
fig = px.treemap(data_frame = rm_score, path=['category'], values='customer_count', color_discrete_sequence=px.colors.qualitative.Pastel1)
fig.show()

[1] 시간의 흐름에 따라 매출, 주문고객수, 주문단가의 추이는 어떻게 달라지는가?
- 매출과 주문고객수는 우상향, 주문단가는 유지

[2] 리텐션 분석
- Month1 리텐션이 최근으로 오며 상승 중
- 2011-11월에 고객 재방문이 늘었다.

[3] RFM 분석
- '최우수' #최신성, 구매 모두 상당히 높음
- '우수' #최신성, 구매 모두 높음
- '휴면' #최신성, 구매 모두 낮음
- '이탈 방지' #구매는 높으나 최신성은 낮음 -> 다시 불러들어야 함
- '구매 유도' #최신성은 높으나 구매는 낮음 -> 구매를 유도해야 함